# <a id='toc1_'></a>[Comparison: Tool Use With vs Without Structured Outputs](#toc0_)

This notebook compares the output of Cohere's tool use with and without the Structured Outputs feature. It highlights the benefits of using Structured Outputs where the tool calling output is guaranteed to follow the specified schema, thus eliminating the issues of model hallucinations.

We'll examine different scenarios:
- Tool name generation
- Tool parameter generation
- Tool parameter data types generation

First, we make the necessary imports and set up the Cohere client.

We'll also create a couple of helper functions to call the Chat API.

For each scenario, we'll use the `display_results` function to make a few calls without and with structured outputs enabled (via the `strict_tools` parameter).

In [4]:
import requests, time, json
import cohere
import os
co = cohere.ClientV2(os.getenv("COHERE_API_KEY")) 

In [15]:
def get_response(message, tools, strict_tools, model, preamble=None):
    url = "https://api.cohere.ai/v2/chat"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {os.getenv('COHERE_API_KEY')}",
    }

    if preamble is not None:
        messages = [{"role": "system", "content": preamble},
                    {"role": "user", "content": message}]
    else:
        messages = [{"role": "user", "content": message}]
        
    payload = {
        "messages": messages,
        "temperature": 0,
        "stream": False,
        "model": model,
        "strict_tools": strict_tools,
        "tools": tools,
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

In [16]:
def display_results(message, tools, model="command-r-08-2024", preamble=None, num_gen=3):
    print("USER MESSAGE:\n" + message)
    print("-"*30)
    print("TOOLS AVAILABLE:")
    for tool in tools:
        print(tool, "\n" + "-"*10) 
    for strict_tools in [False, True]:
        print("WITH STRUCTURED OUTPUTS:" if strict_tools else "WITHOUT STRUCTURED OUTPUTS")
        print("-"*30)
        for i in range(num_gen):
            print(f"TOOL CALLS (ATTEMPT #{i + 1}):")
            response = get_response(message, tools, strict_tools, model, preamble)
            if isinstance(response, str) and response.startswith("Error"):
                print(response)
            elif 'tool_calls' in response['message']:
                for tc in response['message']['tool_calls']:
                    print(f"Tool name: {tc['function']['name']} | Parameters: {tc['function']['arguments']}")
            print("-"*100)

## <a id='toc1_1_'></a>[Eliminate hallucinations of tool names](#toc0_)

In this example, we demonstrate how structured outputs help prevent tool name hallucinations.

When making the API call without structured outputs (strict_tools=False), the model returns hallucinated tool names that don't match our defined tools, resulting in 422 errors.

However, when using structured outputs (strict_tools=True), the model is constrained to only generate the explicitly defined tool names, leading to successful tool calls with valid tool names.

In [17]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "TestObjectGraphAfterGC_doTesting",
                "description": "Executes a test that allocates an object graph based on the provided test case data, runs garbage collection, checks the object graph references, and verifies specific entries in the garbage collector log.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "testcaseData": {
                            "type": "string",
                            "description": "The data for the test case to allocate the object graph. This is Java String type parameter in string representation."
                        },
                        "doGC": {
                            "type": "string",
                            "description": "A Runnable that triggers garbage collection. This parameter can be of any type of Java object in string representation."
                        },
                        "checker": {
                            "type": "string",
                            "description": "A Consumer that checks the object references after garbage collection. This parameter can be of any type of Java object in string representation."
                        },
                        "gcLogName": {
                            "type": "string",
                            "description": "The name of the garbage collector log file. This is Java String type parameter in string representation."
                        },
                        "shouldContain": {
                            "type": "string",
                            "description": "A list of strings that should be present in the garbage collector log. This is Java ArrayList type parameter in string representation. The list elements are of type String; they are not in string representation."
                        },
                        "shouldNotContain": {
                            "type": "string",
                            "description": "A list of strings that should not be present in the garbage collector log. This is Java ArrayList type parameter in string representation. The list elements are of type String; they are not in string representation."
                        }
                    },
                    "required": ["testcaseData"]
                }
            }
        }
    ]

message = "How can I perform a garbage collection test using the data from the 'humongous-test-case.json', execute a custom garbage collector, verify the object references using the `referenceChecker` function, and analyze the garbage collector log named 'gc-analysis.log' to ensure it contains 'GC pause' but does not contain 'OutOfMemoryError'?"

display_results(message, tools, model="command-r-08-2024")

USER MESSAGE:
How can I perform a garbage collection test using the data from the 'humongous-test-case.json', execute a custom garbage collector, verify the object references using the `referenceChecker` function, and analyze the garbage collector log named 'gc-analysis.log' to ensure it contains 'GC pause' but does not contain 'OutOfMemoryError'?
------------------------------
TOOLS AVAILABLE:
{'type': 'function', 'function': {'name': 'TestObjectGraphAfterGC_doTesting', 'description': 'Executes a test that allocates an object graph based on the provided test case data, runs garbage collection, checks the object graph references, and verifies specific entries in the garbage collector log.', 'parameters': {'type': 'object', 'properties': {'testcaseData': {'type': 'string', 'description': 'The data for the test case to allocate the object graph. This is Java String type parameter in string representation.'}, 'doGC': {'type': 'string', 'description': 'A Runnable that triggers garbage coll

## <a id='toc1_2_'></a>[Eliminate hallucinations of tool parameters](#toc0_)

In this example, we demonstrate how structured outputs help prevent tool parameter hallucinations.

When making the API call without structured outputs (strict_tools=False), the model returns hallucinated a tool name (`_from`) when it should be `_fromj8K2mN9pQ4rS7vW`.

However, when using structured outputs (`strict_tools=True`), the model is constrained to only generate the explicitly defined tool parameters, leading to successful tool calls with valid parameter names.

In [57]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "Trains_1_GetTrainTickets",
            "description": "Reserve tickets for a train journey by providing journey details, the number of passengers, and seating class preferences.",
            "parameters": {
                "type": "object",
                "properties": {
                    "_from": {
                        "type": "string",
                        "description": "The departure city for the train journey, in the format of 'City, State', such as 'Los Angeles, CA'.",
                    },
                    "to": {
                        "type": "string",
                        "description": "The arrival city for the train journey, in the format of 'City, State', such as 'New York, NY'.",
                    },
                    "date_of_journey": {
                        "type": "string",
                        "description": "The date of the train journey in the format 'YYYY-MM-DD', such as '2023-04-15'.",
                    },
                    "journey_start_time": {
                        "type": "string",
                        "description": "The start time of the train journey in 24-hour format 'HH:MM', such as '14:30'.",
                    },
                    "number_of_adults": {
                        "type": "integer",
                        "description": "The number of adults to reserve train tickets for. Must be between 1 to 5 adults.",
                    },
                    "trip_protection": {
                        "type": "string",
                        "description": "Indicates whether to add trip protection to the reservation for an additional fee. Trip protection provides refund options and assistance in case of trip disruptions.",
                    },
                    "class": {
                        "type": "string",
                        "description": "The fare class for the train reservation. Options include 'Value', 'Flexible', and 'Business'. The default value is: Value",
                    },
                },
                "required": [
                    "_from",
                    "to",
                    "date_of_journey",
                    "journey_start_time",
                    "number_of_adults",
                    "trip_protection",
                    "class",
                ],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "Trains_1_FindTrains",
            "description": "Finds available train options for a specified route on a given date, allowing users to select a preferred travel class.",
            "parameters": {
                "type": "object",
                "properties": {
                    "_fromj8K2mN9pQ4rS7vW": {
                        "type": "string",
                        "description": "The city where the train journey will start.",
                    },
                    "to": {
                        "type": "string",
                        "description": "The destination city for the train journey.",
                    },
                    "date_of_journey": {
                        "type": "string",
                        "description": "The date of the train journey, in the format of 'YYYY-MM-DD'.",
                    },
                    "class": {
                        "type": "string",
                        "description": "The fare class for the train reservation. The default value is: Value",
                    },
                    "number_of_adults": {
                        "type": "integer",
                        "description": "The number of adults to reserve train tickets for. The default value is: 1.",
                    },
                },
                "required": [
                    "_fromj8K2mN9pQ4rS7vW",
                    "to",
                    "date_of_journey",
                    "class",
                    "number_of_adults",
                ],
            },
        },
    },
]

message = "I'm travelling for work on business and need to find train tickets from Sacramento to Fresno on the 10th of March 2023."

display_results(message, tools, model="command-r")

USER MESSAGE:
I'm travelling for work on business and need to find train tickets from Sacramento to Fresno on the 10th of March 2023.
------------------------------
TOOLS AVAILABLE:
{'type': 'function', 'function': {'name': 'Trains_1_GetTrainTickets', 'description': 'Reserve tickets for a train journey by providing journey details, the number of passengers, and seating class preferences.', 'parameters': {'type': 'object', 'properties': {'_from': {'type': 'string', 'description': "The departure city for the train journey, in the format of 'City, State', such as 'Los Angeles, CA'."}, 'to': {'type': 'string', 'description': "The arrival city for the train journey, in the format of 'City, State', such as 'New York, NY'."}, 'date_of_journey': {'type': 'string', 'description': "The date of the train journey in the format 'YYYY-MM-DD', such as '2023-04-15'."}, 'journey_start_time': {'type': 'string', 'description': "The start time of the train journey in 24-hour format 'HH:MM', such as '14:30

## <a id='toc1_3_'></a>[Make sure all tool parameters have the expected data type](#toc0_)

Without structured outputs, the model may generate tool calls that don't respect parameter types defined in the tool schema.

Here, when asked to save game progress with `stage` as integer', the tool call is generated with `stage` as a string instead.

With structured outputs enabled, the model is forced to respect parameter types. This time, the tool call is generated with `stage` as an integer.


In [25]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "game_save_progress",
            "description": "Save the current state of a player's game, given the stage, level and game mode.",
            "parameters": {
                "type": "object",
                "properties": {
                    "stage": {
                        "type": "integer",
                        "description": "The current stage in the game the player has reached.",
                    },
                    "mode": {
                        "type": "string",
                        "description": "The game mode. Available modes are easy or hard.",
                    },
                    "level": {
                        "type": "string",
                        "description": "The player's level. The default value is: user",
                    },
                },
                "required": ["stage", "mode", "level"],
            },
        },
    }
]

message = "How to save game progress at stage 7 in easy mode and stage 3 in hard mode? 'stage' should be a string."

display_results(message, tools, model="command-r-08-2024")

USER MESSAGE:
How to save game progress at stage 7 in easy mode and stage 3 in hard mode? 'stage' should be a string.
------------------------------
TOOLS AVAILABLE:
{'type': 'function', 'function': {'name': 'game_save_progress', 'description': "Save the current state of a player's game, given the stage, level and game mode.", 'parameters': {'type': 'object', 'properties': {'stage': {'type': 'integer', 'description': 'The current stage in the game the player has reached.'}, 'mode': {'type': 'string', 'description': 'The game mode. Available modes are easy or hard.'}, 'level': {'type': 'string', 'description': "The player's level. The default value is: user"}}, 'required': ['stage', 'mode', 'level']}}} 
----------
WITHOUT STRUCTURED OUTPUTS
------------------------------
TOOL CALLS (ATTEMPT #1):
Tool name: game_save_progress | Parameters: {"level":"user","mode":"easy","stage":"7"}
Tool name: game_save_progress | Parameters: {"level":"user","mode":"hard","stage":"3"}
--------------------

## Conclusion

The examples above demonstrate how Structured Outputs help enforce constraints in tool calls, which eliminates hallucinations in:
- tool names
- tool parameter names
- tool parameter data types

Another scenario not explicitly shown here is Structured Outputs guarantees that the list of `required` parameters listed in the schema will be generated.